In [ ]:
import pyVoxelFEM
import MeshFEM, mesh
import numpy as np
import matplotlib.pyplot as plt
from tri_mesh_viewer import QuadHexViewer
import cyipopt as ipopt
import time
import copy
import benchmark

# Helpers
import sys
sys.path.append('./helpers')
from ipopt_helpers import initializeTensorProductSimulator, problemObjectWrapper, initializeIpoptProblem

# TO problem DEMO

In [ ]:
import parallelism, psutil
parallelism.set_max_num_tbb_threads(psutil.cpu_count(logical=False))
parallelism.set_gradient_assembly_num_threads(min(psutil.cpu_count(logical=False), 8))

In [ ]:
# 2D or 3D?
DIM = 3
useMultigrid = True

# Material and "boundary conditions" (can be imposed also inside the domain) are read from file
MATERIAL_PATH = '../examples/materials/B9Creator.material'
if DIM == 2:
    BC_PATH = '../examples/bcs/cantilever_flexion_E.bc'     # 2D cantilever configuration
elif DIM == 3:
    BC_PATH = '../examples/bcs/3D/cantilever_flexion_E.bc'  # 3D cantilever configuration

# Simulator
if DIM == 2:
    orderFEM = [1, 1]
    domainCorners = [[0, 0], [2, 1]]
    #gridDimensions = [192, 96]
    #gridDimensions = [384, 192]
    gridDimensions = [768, 384]
    #gridDimensions = [1600, 800]
elif DIM == 3:
    orderFEM = [1, 1, 1]
    domainCorners = [[0, 0, 0], [2, 1, 1]]
    #gridDimensions = [128, 64, 64]
    gridDimensions = [256, 128, 128]
    #gridDimensions = [512, 256, 128]
E0 = 1
Emin = 1e-4
SIMPExponent = 3

# Constraints
maxVolume = 0.6
constraints = [pyVoxelFEM.TotalVolumeConstraint(maxVolume)]

# Filters: comment a line to remove the corresponding filter
filters = [
    pyVoxelFEM.SmoothingFilter(),
    pyVoxelFEM.ProjectionFilter(),
#     pyVoxelFEM.LangelaarFilter()
]

# Topology Optimization problem
uniformDensity = maxVolume

In [ ]:
benchmark.reset()
# Initializations
tps = initializeTensorProductSimulator(
    orderFEM, domainCorners, gridDimensions, uniformDensity, E0, Emin, SIMPExponent, MATERIAL_PATH, BC_PATH
)
objective = pyVoxelFEM.MultigridComplianceObjective(tps.multigridSolver(3)) if useMultigrid \
            else pyVoxelFEM.ComplianceObjective(tps)
benchmark.report()

In [ ]:
top = pyVoxelFEM.TopologyOptimizationProblem(tps, objective, constraints, filters) # TO problem
nonLinearProblem, history = initializeIpoptProblem(top)                            # Wrap problem for the optimizer
x0 = tps.getDensities()                                                            # Starting guess (uniform)
top.setVars(x0);

In [ ]:
if useMultigrid:
    # Configure multigrid objective
    objective.tol = 1e-4
    objective.mgIterations = 1
    objective.mgSmoothingIterations = 2
    objective.fullMultigrid = True

In [ ]:
benchmarkSolves = False
if useMultigrid and benchmarkSolves:
    mg = tps.multigridSolver(2)
    benchmark.reset()
    mg.preconditionedConjugateGradient(np.zeros_like(tps.buildLoadVector()), tps.buildLoadVector(), maxIter=100, tol=1e-9, mgSmoothingIterations=2, mgIterations=1, fullMultigrid = True)
    benchmark.report()
    
    #tps.clearCachedElementStiffness()
    #benchmark.reset()
    #tps.solve(tps.buildLoadVector())
    #benchmark.report()

In [ ]:
# Finite difference validation
import fd_validation, benchmark

class FDValidationWrapper:
    def __init__(self, top): self.top = top
    def numVars(self): return top.numVars()
    def getVars(self): return top.getVars()
    def setVars(self, x): self.top.setVars(x)
    def energy(self): return self.top.evaluateObjective()
    def gradient(self): return self.top.evaluateObjectiveGradient()
    def energy(self): return self.top.evaluateConstraints()[0]
    def gradient(self): return self.top.evaluateConstraintsJacobian()[0]

# fd_validation.gradConvergencePlot(FDValidationWrapper(top))

## Run optimization

In [ ]:
max_iters = 100
algorithm = 'OC' # 'OC' or 'LBFGS'

In [ ]:
if (algorithm == 'OC'):
    oco = pyVoxelFEM.OCOptimizer(top)
    benchmark.reset()
    for i in range(max_iters):
        history.density.append(top.getDensities())
        oco.step()
    benchmark.report()
elif (algorithm == 'LBFGS'):
    # Stopping criteria
    nonLinearProblem.addOption('max_iter', 200)
    nonLinearProblem.addOption('tol', 1e-10)

    # Optimize
    benchmark.reset()
    x0, info = nonLinearProblem.solve(x0)
    benchmark.report()
else: raise Exception('Unknown algorithm')

In [ ]:
benchmark.report()

In [ ]:
# Domain view 
view = QuadHexViewer(*tps.getMesh(), scalarField=tps.getDensities())

In [ ]:
view.show()

In [ ]:
# Dynamically update view using optimization history
for density in history.density:
    view.update(scalarField=density)
    time.sleep(0.05)

In [ ]:
def updateSimulation(f = -1):
    u = tps.solveWithImposedLoads()
    m = tri_mesh_viewer.QuadHexMeshWrapper(*tps.getMesh())
    X = m.V.copy()
    m.V = np.array(X + 0.25 * np.pad(u, [(0, 0), (0, 1)]), dtype=np.float32)
    view.update(mesh=m, vectorField=np.pad(Ku, [(0, 0), (0, 1)]), scalarField=tps.getDensities())
    if (f >= 0):
        orender = view.offscreenRenderer(2048, 2048)
        orender.transparentBackground = False
        orender.render()
        orender.save(f'frame_{f}.png')

In [ ]:
for f, alpha in enumerate(np.linspace(0, 1.0, 60)):
    for i, d in enumerate(densities.ravel()):
        tps.setElementDensity(i, -((1 - alpha) * d +  alpha * np.round(d)))
    updateSimulation(f)

In [ ]:
for i, d in enumerate(densities.ravel()):
    tps.setElementDensity(i, -d)
updateSimulation()

In [ ]:
m = tri_mesh_viewer.QuadHexMeshWrapper(*tps.getMesh())
X = m.V.copy()

In [ ]:
m.V = np.array(X + 0.25 * np.pad(u, [(0, 0), (0, 1)]), dtype=np.float32)

In [ ]:
view.update(mesh=m, vectorField=np.pad(Ku, [(0, 0), (0, 1)]), scalarField=tps.getDensities())

### Modify filters
Run the cells below to change filters parameters. Then, restart optimization.

In [ ]:
# Modify smoothing radius (in all smoothing filters of the filters chain)
newSmoothingRadius = 0.5
for filt in filters:
    if type(filt) == pyVoxelFEM.SmoothingFilter:
        filt.radius = newSmoothingRadius

In [ ]:
# Modify projection steepness (in all projection filters of the filters chain)
newBeta = 4
for filt in filters:
    if type(filt) == pyVoxelFEM.ProjectionFilter:
        filt.beta = newBeta

### Export to `.msh`

In [ ]:
BASE_PATH='out'
PATH=BASE_PATH + '.msh'

In [ ]:
# Export final density to .msh
mfw = mesh.MSHFieldWriter(PATH, *tps.getMesh())
mfw.addField('density', tps.getDensities())

In [ ]:
# Export whole history to a single .msh (for manual visualization)
mfw = mesh.MSHFieldWriter(PATH, *tps.getMesh())
for iteration, density in enumerate(history.density):
    mfw.addField('iter' + str(iteration), density)

In [ ]:
# Export whole history to multiple .msh files (for animation recording)
for iteration, density in enumerate(history.density):
    mfw = mesh.MSHFieldWriter(BASE_PATH + 'iter' + str(iteration).zfill(4) + '.msh', *tps.getMesh())
    mfw.addField('density', density)

### Plot metrics history

In [ ]:
history.plotObjective()

In [ ]:
history.plotNondiscreteness()